In [5]:
# Install required packages
!pip install tensorflow
!pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite-runtime
!pip install efficientnet
!pip install imgaug albumentations

Looking in indexes: https://pypi.org/simple, https://google-coral.github.io/py-repo/


ERROR: Could not find a version that satisfies the requirement tflite-runtime (from versions: none)
ERROR: No matching distribution found for tflite-runtime


In [41]:
import tensorflow as tf
from efficientnet.tfkeras import EfficientNetB0
import logging
import datetime
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import os
import json
import numpy as np

In [46]:
import subprocess
import sys

def check_cuda():
    print("Python version:", sys.version)
    print("\nCUDA version:")
    try:
        nvcc_output = subprocess.check_output("nvcc --version", shell=True)
        print(nvcc_output.decode())
    except:
        print("CUDA not found or nvcc not in PATH")
    
    print("\nNVIDIA-SMI:")
    try:
        nvidia_smi_output = subprocess.check_output("nvidia-smi", shell=True)
        print(nvidia_smi_output.decode())
    except:
        print("nvidia-smi not found")

check_cuda()

Python version: 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]

CUDA version:
CUDA not found or nvcc not in PATH

NVIDIA-SMI:
Fri Jun 13 02:36:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.52                 Driver Version: 576.52         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   60C    P5              6W /   74W |     202MiB / 

In [29]:
!pip install tensorflow==2.18.0

  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
Using cached ml_dtypes-0.4.1-cp312-cp312-win_amd64.whl (127 kB)
Using cached tensorboard-2.18.0-py3-none-any.whl (5.5 MB)
  Attempting uninstall: ml-dtypes
    Found existing installation: ml_dtypes 0.5.1
    Uninstalling ml_dtypes-0.5.1:
      Successfully uninstalled ml_dtypes-0.5.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.19.0
    Uninstalling tensorboard-2.19.0:
      Successfully uninstalled tensorboard-2.19.0


In [30]:
# Install the latest compatible cuDNN version
!pip install nvidia-cudnn-cu12==9.10.2.21

   ---------------------------------------- 0.0/693.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/693.0 MB 12.7 MB/s eta 0:00:55
   ---------------------------------------- 2.1/693.0 MB 5.1 MB/s eta 0:02:16
   ---------------------------------------- 2.1/693.0 MB 5.1 MB/s eta 0:02:16
   ---------------------------------------- 3.1/693.0 MB 4.3 MB/s eta 0:02:41
   ---------------------------------------- 3.1/693.0 MB 4.3 MB/s eta 0:02:41
   ---------------------------------------- 4.2/693.0 MB 3.4 MB/s eta 0:03:23
   ---------------------------------------- 4.2/693.0 MB 3.4 MB/s eta 0:03:23
   ---------------------------------------- 5.0/693.0 MB 2.9 MB/s eta 0:03:55
   ---------------------------------------- 5.2/693.0 MB 3.0 MB/s eta 0:03:47
   ---------------------------------------- 6.3/693.0 MB 3.0 MB/s eta 0:03:52
   ---------------------------------------- 6.6/693.0 MB 2.8 MB/s eta 0:04:06
   ---------------------------------------- 7.3/693.0 MB 3.1 MB/s eta 

In [43]:
import tensorflow as tf
import os

# Set environment variables
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

# Print TensorFlow information
print("TensorFlow version:", tf.__version__)
print("GPU Available: ", tf.config.list_physical_devices('GPU'))
print("GPU Device Name: ", tf.test.gpu_device_name())

# Configure GPU memory growth
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        for device in physical_devices:
            tf.config.experimental.set_memory_growth(device, True)
        print("GPU memory growth enabled")
    except RuntimeError as e:
        print(f"Error setting GPU memory growth: {e}")

TensorFlow version: 2.18.0
GPU Available:  []
GPU Device Name:  


In [44]:
def check_system_requirements():
    import platform
    import psutil
    
    logging.info(f"Python version: {platform.python_version()}")
    logging.info(f"TensorFlow version: {tf.__version__}")
    logging.info(f"Number of CPUs: {psutil.cpu_count()}")
    logging.info(f"Total RAM: {psutil.virtual_memory().total / (1024**3):.2f} GB")
    
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        logging.info(f"Number of GPUs: {len(gpus)}")
        for gpu in gpus:
            logging.info(f"GPU: {gpu}")
    else:
        logging.warning("No GPU found. Training will be slow on CPU.")

# Run system check
check_system_requirements()

# GPU Memory Management
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            # Set memory limit to 3GB to avoid OOM errors
            tf.config.experimental.set_virtual_device_configuration(
                gpu,
                [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3072)]
            )
        logging.info("GPU memory growth enabled")
    except RuntimeError as e:
        logging.error(f"Error setting GPU memory growth: {str(e)}")

In [ ]:
class_names = [
    'ascaris_lumbricoides',
    'trichuris_trichiura',
    'hookworms',
    'enterobius_vermicularis',
    'giardia_lamblia'
]

In [ ]:
def setup_logging():
    log_dir = 'logs'
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    log_file = f'{log_dir}/training_{timestamp}.log'
    
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler()
        ]
    )
    return log_file

def preprocess_image(img):
    try:
        img = img.astype(np.float32) / 255.0
        img = tf.image.per_image_standardization(img)
        return img
    except Exception as e:
        logging.error(f"Error in preprocessing: {str(e)}")
        return None

def save_model_version(model, version_info):
    version_dir = 'model_versions'
    if not os.path.exists(version_dir):
        os.makedirs(version_dir)
    
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    version_path = f'{version_dir}/model_v{timestamp}'
    os.makedirs(version_path)
    
    model.save(f'{version_path}/model.h5')
    with open(f'{version_path}/version_info.json', 'w') as f:
        json.dump(version_info, f)
    
    logging.info(f"Model saved to {version_path}")
    return version_path

In [ ]:
class LoggingCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        logging.info(f"Epoch {epoch+1} - loss: {logs['loss']:.4f} - accuracy: {logs['accuracy']:.4f} - val_loss: {logs['val_loss']:.4f} - val_accuracy: {logs['val_accuracy']:.4f}")

In [ ]:
def validate_data_directories():
    train_dir = './parasites/train'
    val_dir = './parasites/val'
    
    if not os.path.exists(train_dir) or not os.path.exists(val_dir):
        raise ValueError("Training or validation directory not found")
    
    for class_name in class_names:
        train_class_dir = os.path.join(train_dir, class_name)
        val_class_dir = os.path.join(val_dir, class_name)
        
        train_count = len(os.listdir(train_class_dir))
        val_count = len(os.listdir(val_class_dir))
        
        logging.info(f"{class_name}: {train_count} training images, {val_count} validation images")
        
        if train_count < 10:
            logging.warning(f"Low number of training images for {class_name}: {train_count}")
        if val_count < 5:
            logging.warning(f"Low number of validation images for {class_name}: {val_count}")

# Run validation
try:
    validate_data_directories()
    logging.info("Data validation completed successfully")
except Exception as e:
    logging.error(f"Data validation failed: {str(e)}")
    raise

In [ ]:
# Initialize logging
log_file = setup_logging()
logging.info("Starting training process")

# Create model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3))
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dropout(0.3)(x)
output = tf.keras.layers.Dense(5, activation='softmax')(x)
model = tf.keras.Model(inputs=base_model.input, outputs=output)

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=3,
            min_lr=1e-6
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ModelCheckpoint(
            'best_model.h5',
            monitor='val_accuracy',
            save_best_only=True
        ),
        LoggingCallback(),
        tf.keras.callbacks.TensorBoard(log_dir='./logs')
    ],
    verbose=1
)

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    './parasites/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    './parasites/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
def evaluate_model(model, validation_generator, class_names):
    try:
        y_pred = []
        y_true = []
        
        for i in range(len(validation_generator)):
            x, y = validation_generator[i]
            preds = model.predict(x)
            y_pred.extend(np.argmax(preds, axis=1))
            y_true.extend(np.argmax(y, axis=1))
        
        cm = confusion_matrix(y_true, y_pred)
        
        plt.figure(figsize=(10,8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=class_names,
                   yticklabels=class_names)
        plt.title('Confusion Matrix')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.xticks(rotation=45)
        plt.yticks(rotation=45)
        plt.tight_layout()
        plt.savefig('confusion_matrix.png')
        plt.close()
        
        report = classification_report(y_true, y_pred, 
                                    target_names=class_names,
                                    output_dict=True)
        logging.info("Classification Report:")
        logging.info(json.dumps(report, indent=4))
        
        system_accuracy = np.mean(np.array(y_true) == np.array(y_pred))
        logging.info(f"System Accuracy: {system_accuracy:.4f}")
        
        return report, system_accuracy
        
    except Exception as e:
        logging.error(f"Error in evaluation: {str(e)}")
        return None, None

In [ ]:
# Save model version
version_info = {
    'model_type': 'EfficientNetB0',
    'input_shape': (224, 224, 3),
    'num_classes': 5,
    'training_date': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'final_accuracy': history.history['accuracy'][-1],
    'final_val_accuracy': history.history['val_accuracy'][-1]
}
model_version_path = save_model_version(model, version_info)

# Run evaluation
report, system_accuracy = evaluate_model(model, validation_generator, class_names)

In [ ]:
# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS_INT8,
    tf.lite.OpsSet.TFLITE_BUILTINS
]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_model = converter.convert()

# Save TFLite model
with open('parasite_classifier.tflite', 'wb') as f:
    f.write(tflite_model)

# Code for the raspbery pi app

In [ ]:
import cv2
import numpy as np
from tflite_runtime.interpreter import Interpreter
import logging
import os
from datetime import datetime

# Setup logging for Raspberry Pi
def setup_logging():
    log_dir = 'logs'
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    log_file = f'{log_dir}/prediction_{timestamp}.log'
    
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler()
        ]
    )
    return log_file

def preprocess_image(img):
    try:
        img = img.astype(np.float32) / 255.0
        return img
    except Exception as e:
        logging.error(f"Error in preprocessing: {str(e)}")
        return None

def classify_image(image_path, interpreter, input_details, output_details, class_names, threshold=0.7):
    try:
        # Read image
        img = cv2.imread(image_path)
        if img is None:
            logging.error(f"Failed to read image: {image_path}")
            return None, "Failed to read image"
        
        # Resize image
        img = cv2.resize(img, (224, 224))
        
        # Preprocess image
        img = preprocess_image(img)
        if img is None:
            return None, "Error in preprocessing"
        
        # Prepare input tensor
        input_data = np.expand_dims(img, axis=0)
        
        # Set input tensor
        interpreter.set_tensor(input_details[0]['index'], input_data)
        
        # Run inference
        interpreter.invoke()
        
        # Get output tensor
        output_data = interpreter.get_tensor(output_details[0]['index'])[0]
        
        # Get prediction
        max_prob = np.max(output_data)
        if max_prob >= threshold:
            class_id = np.argmax(output_data)
            return class_names[class_id], max_prob
        else:
            return "No confident prediction", max_prob
            
    except Exception as e:
        logging.error(f"Error in classification: {str(e)}")
        return None, "Error in classification"

def main():
    # Setup logging
    log_file = setup_logging()
    logging.info("Starting parasite classification")
    
    # Class names
    class_names = [
        'ascaris_lumbricoides',
        'trichuris_trichiura',
        'hookworms',
        'enterobius_vermicularis',
        'giardia_lamblia'
    ]
    
    try:
        # Load TFLite model
        interpreter = Interpreter(model_path='parasite_classifier.tflite')
        interpreter.allocate_tensors()
        
        # Get I/O details
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()
        
        # Create directory for captured images if it doesn't exist
        images_dir = 'captured_images'
        if not os.path.exists(images_dir):
            os.makedirs(images_dir)
        
        while True:
            # Wait for user input
            input("Press Enter to capture and classify an image (or Ctrl+C to exit)...")
            
            # Generate timestamp for image filename
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            image_path = f'{images_dir}/parasite_{timestamp}.jpg'
            
            # Capture image using Raspberry Pi camera
            # Note: You'll need to implement the actual camera capture code here
            # This is a placeholder for the camera capture
            # You might want to use picamera2 or other camera libraries
            # For example:
            # with picamera2.Picamera2() as camera:
            #     camera.capture_file(image_path)
            
            logging.info(f"Image captured and saved to: {image_path}")
            
            # Classify the image
            prediction, confidence = classify_image(
                image_path,
                interpreter,
                input_details,
                output_details,
                class_names
            )
            
            if prediction:
                result = f"Prediction: {prediction} (Confidence: {confidence:.2f})"
                logging.info(result)
                print(result)
                
                # Display the image with prediction
                img = cv2.imread(image_path)
                cv2.putText(
                    img,
                    result,
                    (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1,
                    (0, 255, 0),
                    2
                )
                cv2.imshow('Parasite Classification', img)
                cv2.waitKey(0)
                cv2.destroyAllWindows()
            else:
                logging.error("Classification failed")
                print("Classification failed")
                
    except KeyboardInterrupt:
        logging.info("Program terminated by user")
        print("\nProgram terminated by user")
    except Exception as e:
        logging.error(f"Unexpected error: {str(e)}")
        print(f"Unexpected error: {str(e)}")
    finally:
        cv2.destroyAllWindows()

if __name__ == "__main__":
    main()